In [16]:
import pandas as pd
import numpy as np
import requests

#!pip install foursquare
import foursquare

from bs4 import BeautifulSoup
#!pip install googlemaps
import googlemaps
#!pip install geocoder
import geocoder

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library


In [5]:
# Web scrape Rent Data for Palo Alto
source1 = requests.get('https://www.rentcafe.com/average-rent-market-trends/us/ca/santa-clara-county/palo-alto/').text
soup1 = BeautifulSoup(source1, 'html5lib')
table1=soup1.find('table',class_='market-trends',id='MarketTrendsAverageRentTable')
rows1 =table1.find_all('tr', class_="current-row")
rent_palo_alto_dict={}
for i in range(len(rows1)):
    Neighborhood=rows1[i].th.text
    Rent=rows1[i].td.text
    rent_palo_alto_dict[Neighborhood]=Rent
    
columns = ['Neighborhood','Rent']
palo_alto_data = pd.DataFrame(columns=columns)
for k,v in rent_palo_alto_dict.items():
    Neighborhood=k
    Rent=v
    palo_alto_data=palo_alto_data.append({'Neighborhood':Neighborhood,'Rent':Rent},ignore_index=True)

palo_alto_data.head()

,Neighborhood,Rent
0,Triple El,"$2,693"
1,The Greenhouse,"$2,693"
2,Palo Verde,"$2,693"
3,Leland Manor,"$2,693"
4,Garland Park,"$2,693"


In [6]:
# Convert Rent data into interger for future analysis
palo_alto_data['Rent']=palo_alto_data['Rent'].str.replace('$','')
palo_alto_data['Rent']=palo_alto_data['Rent'].str.replace(',','')
palo_alto_data['Rent']=palo_alto_data['Rent'].astype(int)
palo_alto_data.head()

,Neighborhood,Rent
0,Triple El,2693
1,The Greenhouse,2693
2,Palo Verde,2693
3,Leland Manor,2693
4,Garland Park,2693


In [100]:
# The code was removed by Watson Studio for sharing.

In [33]:
# Obtain Coordinate for each Neighborhood
lat_lng_coords_pa={}
neighborhoods=palo_alto_data['Neighborhood']
for neighborhood in neighborhoods:
    g = geocoder.google('{}, Palo Alto, California'.format(neighborhood),key=key_google)
    while(g.latlng is None):
        g = geocoder.google('{}, Palo Alto, California'.format(neighborhood),key=key_google)
    latitude = g.latlng[0]
    longitude = g.latlng[1]
    lat_lng_coords_pa[neighborhood]={}
    lat_lng_coords_pa[neighborhood]['Latitude']=latitude
    lat_lng_coords_pa[neighborhood]['Longitude']=longitude

In [37]:
# Save Coordinate data into datafram
columns=['Neighborhood','Latitude','Longitude']
pa_geo = pd.DataFrame(columns=columns)

for ind, neighborhood in enumerate(lat_lng_coords_pa):
    latitude= lat_lng_coords_pa[neighborhood]['Latitude']
    longitude= lat_lng_coords_pa[neighborhood]['Longitude']
    pa_geo=pa_geo.append({'Neighborhood':neighborhood,
                           'Latitude':latitude,
                            'Longitude':longitude},ignore_index=True)

pa_geo.head()

,Neighborhood,Latitude,Longitude
0,Triple El,37.442803,-122.129894
1,The Greenhouse,37.419367,-122.102549
2,Palo Verde,37.430655,-122.112530
3,Leland Manor,37.441466,-122.136282
4,Garland Park,37.438903,-122.132074


In [39]:
# Merge Rent with Coordinate
pa_rent_co= palo_alto_data
pa_rent_co = pa_rent_co.join(pa_geo.set_index('Neighborhood'),on='Neighborhood')
pa_rent_co.head()

,Neighborhood,Rent,Latitude,Longitude
0,Triple El,2693,37.442803,-122.129894
1,The Greenhouse,2693,37.419367,-122.102549
2,Palo Verde,2693,37.430655,-122.112530
3,Leland Manor,2693,37.441466,-122.136282
4,Garland Park,2693,37.438903,-122.132074


In [101]:
# The code was removed by Watson Studio for sharing.

In [41]:
# define functions
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [103]:
LIMIT = 50
pa_venues = getNearbyVenues(names=pa_rent_co['Neighborhood'],
                                   latitudes=pa_rent_co['Latitude'],
                                   longitudes=pa_rent_co['Longitude']
                                  )

Triple El
The Greenhouse
Palo Verde
Leland Manor
Garland Park
Embarcadero Oaks
East Palo Alto
Duveneck - St Francis
Charleston Gardens
Adobe Meadow - Meadow Park
Midtown Palo Alto
Walnut Grove
Ventura
St. Claire Gardens
Southgate
South of Midtown
San Alma
Palo Alto Orchards
Palo Alto Gardens
Monroe Park
Mayfield
Greenmeadow
Greendell
Green Acres
Greater Miranda
Fairmeadow
Evergreen Park
Esther Clark Park
Charleston Meadows
University South
Professorville
Old Palo Alto
Downtown North Palo Alto
Crescent Park
Community Center
College Terrace
Barron Park
Palo Alto Hills Golf and Country Club
Foothills Open Space Preserve


In [105]:
print('There are {} uniques categories.'.format(len(pa_venues['Venue Category'].unique())))

There are 152 uniques categories.


In [106]:
# Analyze Each Neighborhood
pa_onehot = pd.get_dummies(pa_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pa_onehot['Neighborhood'] = pa_venues['Neighborhood'] 

# move neighborhood column to the first column
pa_onehot= pa_onehot.set_index('Neighborhood').reset_index()


pa_onehot.head()

,Neighborhood,ATM,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Bagel Shop,Bakery,...,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Greenhouse,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
pa_grouped = pa_onehot.groupby('Neighborhood').mean().reset_index()
pa_grouped.head()

,Neighborhood,ATM,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Bagel Shop,Bakery,...,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Adobe Meadow - Meadow Park,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Barron Park,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Charleston Gardens,0.0,0.0,0.0,0.0,0.000,0.0,0.054054,0.0,0.0,...,0.0,0.0,0.027027,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Charleston Meadows,0.0,0.0,0.0,0.0,0.000,0.0,0.076923,0.0,0.0,...,0.0,0.0,0.000000,0.076923,0.0,0.0,0.0,0.0,0.0,0.0
4,College Terrace,0.0,0.0,0.0,0.0,0.125,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
# Put rent data into dataset
pa_grouped = pa_grouped.join(palo_alto_data.set_index('Neighborhood'), on='Neighborhood')
pa_grouped.head()

,Neighborhood,ATM,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio,Rent
0,Adobe Meadow - Meadow Park,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2693
1,Barron Park,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3859
2,Charleston Gardens,0.0,0.0,0.0,0.0,0.000,0.0,0.054054,0.0,0.0,...,0.0,0.027027,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2693
3,Charleston Meadows,0.0,0.0,0.0,0.0,0.000,0.0,0.076923,0.0,0.0,...,0.0,0.000000,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,3426
4,College Terrace,0.0,0.0,0.0,0.0,0.125,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3859


In [109]:
# KMEAN cluster implementation
kclusters = 3

pa_clustering = pa_grouped.drop(['Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pa_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0, 2, 0, 1, 2, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1], dtype=int32)

In [110]:
# Put top venues into a pandas data frame

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pa_grouped1=pa_grouped.drop('Rent',1)
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = pa_grouped1['Neighborhood']

for ind in np.arange(pa_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pa_grouped1.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adobe Meadow - Meadow Park,Playground,Pizza Place,Dog Run,Pet Store,Burrito Place,Salon / Barbershop,Café,Coffee Shop,Park,Yoga Studio
1,Barron Park,Professional & Other Places,Park,Yoga Studio,Financial or Legal Service,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop
2,Charleston Gardens,Dance Studio,Gym,Automotive Shop,Rental Car Location,Grocery Store,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Science Museum,Sandwich Place
3,Charleston Meadows,Intersection,Pizza Place,Sandwich Place,Sushi Restaurant,Coffee Shop,Park,Korean Restaurant,Automotive Shop,Rental Car Location,Thrift / Vintage Store
4,College Terrace,Park,Gas Station,Mattress Store,Arts & Crafts Store,Chinese Restaurant,Sushi Restaurant,Bank,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


In [112]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pa_merge_venues=neighborhoods_venues_sorted
pa_merge_venues= neighborhoods_venues_sorted.join(pa_rent_co.set_index('Neighborhood'),on='Neighborhood')
pa_merge_venues.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Rent,Latitude,Longitude
0,0,Adobe Meadow - Meadow Park,Playground,Pizza Place,Dog Run,Pet Store,Burrito Place,Salon / Barbershop,Café,Coffee Shop,Park,Yoga Studio,2693,37.421475,-122.114825
1,2,Barron Park,Professional & Other Places,Park,Yoga Studio,Financial or Legal Service,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,3859,37.412468,-122.136532
2,0,Charleston Gardens,Dance Studio,Gym,Automotive Shop,Rental Car Location,Grocery Store,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Science Museum,Sandwich Place,2693,37.419749,-122.105047
3,1,Charleston Meadows,Intersection,Pizza Place,Sandwich Place,Sushi Restaurant,Coffee Shop,Park,Korean Restaurant,Automotive Shop,Rental Car Location,Thrift / Vintage Store,3426,37.414588,-122.122086
4,2,College Terrace,Park,Gas Station,Mattress Store,Arts & Crafts Store,Chinese Restaurant,Sushi Restaurant,Bank,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,3859,37.422772,-122.150478


In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
print(ys)
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pa_merge_venues['Latitude'], pa_merge_venues['Longitude'], pa_merge_venues['Neighborhood'], pa_merge_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

[array([0, 1, 2]), array([1, 3, 7]), array([ 2,  7, 20])]


In [115]:
# Examine Cluster
pa_merge_venues.loc[pa_merge_venues['Cluster Labels'] == 0, pa_merge_venues.columns[[12] + list(range(2, pa_merge_venues.shape[1]-3))]]

,Rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2693,Playground,Pizza Place,Dog Run,Pet Store,Burrito Place,Salon / Barbershop,Café,Coffee Shop,Park,Yoga Studio
2,2693,Dance Studio,Gym,Automotive Shop,Rental Car Location,Grocery Store,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Science Museum,Sandwich Place
8,2693,Gas Station,Bagel Shop,Chinese Restaurant,Gym,Coffee Shop,Tailor Shop,Shopping Plaza,Bank,Market,Ice Cream Shop
9,2693,Mexican Restaurant,ATM,Bagel Shop,Gas Station,Grocery Store,Gym / Fitness Center,Fast Food Restaurant,Market,Home Service,Library
10,2693,Coffee Shop,Gas Station,Shopping Plaza,Gym,Chinese Restaurant,Bagel Shop,Financial or Legal Service,French Restaurant,Food Truck,Food Court
14,2693,Baseball Field,Furniture / Home Store,Intersection,Yoga Studio,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop
19,2693,Pool,Art Gallery,Art Museum,Garden,Park,Yoga Studio,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
21,2862,Pharmacy,Gym,Mexican Restaurant,Bank,Yoga Studio,Shoe Repair,Food Truck,Café,Skating Rink,Shopping Plaza
27,2693,Intersection,Pool,Food Truck,Gym,Park,Yoga Studio,Financial or Legal Service,Fried Chicken Joint,French Restaurant,Food Court
33,2693,Gym,Automotive Shop,Rental Car Location,Jewish Restaurant,Flower Shop,Fast Food Restaurant,Science Museum,Fried Chicken Joint,Dance Studio,Paper / Office Supplies Store


In [116]:
# Examine Cluster
pa_merge_venues.loc[pa_merge_venues['Cluster Labels'] == 1, pa_merge_venues.columns[[12] + list(range(2, pa_merge_venues.shape[1]-3))]]

,Rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,3426,Intersection,Pizza Place,Sandwich Place,Sushi Restaurant,Coffee Shop,Park,Korean Restaurant,Automotive Shop,Rental Car Location,Thrift / Vintage Store
11,3426,Café,Intersection,Plaza,Food Truck,Financial or Legal Service,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court,Flower Shop
12,3426,French Restaurant,Mexican Restaurant,Italian Restaurant,Bank,Organic Grocery,Japanese Restaurant,Sandwich Place,Sushi Restaurant,Mediterranean Restaurant,Coffee Shop
13,3426,Intersection,Indian Restaurant,Dog Run,Playground,Café,Park,Yoga Studio,Flower Shop,Fried Chicken Joint,French Restaurant
15,3426,Garden,Intersection,Pool,Theater,Dog Run,Financial or Legal Service,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
16,3426,Rental Car Location,Hotel,American Restaurant,Park,Pharmacy,Gym / Fitness Center,Fast Food Restaurant,French Restaurant,Food Truck,Food Court
17,3426,Dance Studio,Grocery Store,Automotive Shop,Nail Salon,Shopping Plaza,Chinese Restaurant,Soccer Field,Pizza Place,Science Museum,Sandwich Place
18,3426,Dance Studio,Bookstore,Pizza Place,Deli / Bodega,Pool,Coffee Shop,Salon / Barbershop,Sandwich Place,Science Museum,Chinese Restaurant
20,3426,Coffee Shop,American Restaurant,Grocery Store,Spa,Bakery,Burger Joint,Yoga Studio,Mobile Phone Shop,Clothing Store,Rental Car Location
22,3426,Hotel,Gym / Fitness Center,Spa,Park,Chinese Restaurant,Bistro,Food Truck,Seafood Restaurant,Music Store,Motel


In [117]:
# Examine Cluster
pa_merge_venues.loc[pa_merge_venues['Cluster Labels'] == 2, pa_merge_venues.columns[[12] + list(range(2, pa_merge_venues.shape[1]-3))]]

,Rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,3859,Professional & Other Places,Park,Yoga Studio,Financial or Legal Service,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop
4,3859,Park,Gas Station,Mattress Store,Arts & Crafts Store,Chinese Restaurant,Sushi Restaurant,Bank,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
5,3820,Park,Theater,Performing Arts Venue,Pool,Tennis Court,Campground,Food Court,Nightlife Spot,Café,Library
6,3820,Pizza Place,Park,Yoga Studio,Financial or Legal Service,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop
7,3820,Park,Yoga Studio,Tapas Restaurant,Pastry Shop,Creperie,Coffee Shop,New American Restaurant,Chinese Restaurant,Burger Joint,Mexican Restaurant
23,3820,Garden,Rest Area,Israeli Restaurant,Intersection,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,Financial or Legal Service
25,4086,Golf Course,Financial or Legal Service,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flower Shop,Yoga Studio
28,3820,Yoga Studio,Elementary School,Park,Massage Studio,Garden,Historic Site,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
34,3820,Coffee Shop,Electronics Store,French Restaurant,Frozen Yogurt Shop,Greek Restaurant,Shoe Store,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Salad Place
